In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
df = pd.read_csv("../../data/raw/news/newsapiorg_labelled.csv", index_col=0)
df.head()

,title,publishedAt,url,content,vader_neg,vader_neu,vader_pos,vader_comp,vader_label,label,include,relevant
Column1,,,,,,,,,,,,
1,Key remarks from Russian c.bank governor after...,2022-02-11T12:26:00Z,https://www.reuters.com/markets/currencies/key...,"MOSCOW, Feb 11 (Reuters) - The Russian Central...",0.000,1.000,0.0,0.0000,0,0,1,0
2,LIVE MARKETS Forex: worrying about an inverted...,2022-02-18T12:33:00Z,https://www.reuters.com/markets/stocks/live-ma...,Feb 18 - Welcome to the home for real-time cov...,0.211,0.789,0.0,-0.3400,-1,-1,1,1
3,Dollar takes a break from this week's slide ah...,2022-02-03T02:18:00Z,https://www.reuters.com/markets/europe/dollar-...,"SINGAPORE, Feb 3 (Reuters) - The dollar found ...",0.000,1.000,0.0,0.0000,0,-1,1,1
4,Euro bounce pauses ahead of U.S. inflation - R...,2022-02-08T01:16:00Z,https://www.reuters.com/business/euro-bounce-p...,"SINGAPORE, Feb 8 (Reuters) - A resurgent euro ...",0.000,1.000,0.0,0.0000,0,1,1,1
5,Dollar near two-week high amid jitters over ha...,2022-01-25T01:52:00Z,https://www.reuters.com/markets/europe/dollar-...,"TOKYO, Jan 25 (Reuters) - The safe-haven U.S. ...",0.197,0.803,0.0,-0.4019,-1,-1,1,1


In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [4]:
import sklearn
from sklearn.model_selection import train_test_split
train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(df.title, df.label, test_size = 0.2)
train = pd.concat([train_inputs, train_labels], axis = 1)
test = pd.concat([valid_inputs, valid_labels], axis = 1)
train.head()

,title,label
Column1,,
780,"Biden sends DNC dollars to House, Senate commi...",0
1608,"Russian industry targeted, not consumers, if B...",0
1431,Rate hike bets keep US dollar bid,1
845,Links 1/26/2022,0
1508,Xtrackers by DWS Launches ETF Providing Dynami...,0


In [5]:
def convert_data_to_examples(train, test, title, label): 
  train_InputExamples = train.apply(
    lambda x: InputExample(guid=None, text_a = x[title], text_b = None, label = x[label]), axis = 1)

  validation_InputExamples = test.apply(
    lambda x: InputExample(guid=None, text_a = x[title], text_b = None, label = x[label]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, 'title', 'label')

In [6]:
def convert_examples_to_tf_dataset(examples, tokenizer):
    features = []

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a, 
            add_special_tokens=True,
            max_length=512, 
            return_token_type_ids=True, 
            return_attention_mask=True, 
            padding=True,
            truncation=True
        )
        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],input_dict["token_type_ids"], input_dict['attention_mask'])
        features.append(
            InputFeatures(
                input_ids=input_ids, 
                attention_mask=attention_mask, 
                token_type_ids=token_type_ids, 
                label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [7]:
train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(100).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(100)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)